In [12]:
import pandas as pd #Analysis 
import numpy as np #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization

# 한글설정
from matplotlib import font_manager, rc 
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

### 데이터 불러오기

In [13]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sub = pd.read_csv('../data/sample_submission.csv')
age = pd.read_csv('../data/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

### 데이터 프레임 컬러명 수정(긴 이름들이 있어서)

In [14]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']

### 데이터 오류로 인한 데이터 제외

##### 1. 테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지입니다.

In [15]:
len(train['단지코드'].unique())

423

##### train에서 해당 데이터를 확인

In [16]:
train.loc[((train['단지코드'] == 'C2675')|
          (train['단지코드'] == 'C2335')|
          (train['단지코드'] == 'C1327')),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


##### test에서 해당 데이터 확인

In [17]:
test.loc[((test['단지코드'] == 'C2675')|
          (test['단지코드'] == 'C2335')|
          (test['단지코드'] == 'C1327')),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
579,C2675,512,아파트,경기도,국민임대,36.65,130,9.0,A,18476000,154790,0.0,3.0,1016.0
580,C2675,512,아파트,경기도,국민임대,46.90,44,9.0,A,34082000,232200,0.0,3.0,1016.0
581,C2675,512,아파트,경기도,국민임대,46.90,80,9.0,A,34082000,232200,0.0,3.0,1016.0
582,C2675,512,아파트,경기도,국민임대,46.90,50,9.0,A,34082000,232200,0.0,3.0,1016.0
583,C2675,512,아파트,경기도,국민임대,46.90,88,9.0,A,34082000,232200,0.0,3.0,1016.0
584,C2675,512,아파트,경기도,국민임대,47.40,30,9.0,A,34082000,232200,0.0,3.0,1016.0
585,C2675,512,아파트,경기도,국민임대,51.93,75,9.0,A,43819000,275900,0.0,3.0,1016.0
586,C2675,512,아파트,경기도,국민임대,51.93,15,9.0,A,43819000,275900,0.0,3.0,1016.0
891,C1327,98,아파트,대전광역시,행복주택,16.83,30,13.0,J,18800000,94000,NaN,17.0,98.0
892,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000,142000,NaN,17.0,98.0


##### test에서 3개의 코드 데이터 삭제

In [18]:
test = test.loc[~((test['단지코드'] == 'C2675')|
               (test['단지코드'] == 'C2335')|
               (test['단지코드'] == 'C1327')),:]

##### 확인

In [19]:
test.loc[((test['단지코드'] == 'C2675')|
          (test['단지코드'] == 'C2335')|
          (test['단지코드'] == 'C1327')),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


##### 2. 동일한 단지에 코드가 2개로 부여된 단지 코드 (3쌍) : ['C2085', 'C1397'], ['C2431', 'C1649'], ['C1036', 'C2675']
##### (참고 사항) 주차면수는 하나의 단지임을 전제로 산정된 것이고 총세대수는 두 개 단지의 합계입니다. 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치

In [48]:
train.loc[train['단지코드'] == 'C2085', '총세대수'] = 1339
train.loc[train['단지코드'] == 'C1397', '총세대수'] = 1339

In [49]:
c = train.loc[:, ['단지코드','총세대수']]
c

,단지코드,총세대수
0,C2483,900
1,C2483,900
2,C2483,900
3,C2483,900
4,C2483,900
...,...,...
2947,C2532,239
2948,C2532,239
2949,C2532,239
2950,C2532,239


In [41]:
print(train.loc[ train['단지코드']=='C2085', : ].shape)
print(train.loc[ train['단지코드']=='C1397', : ].shape)

(8, 15)
(6, 15)


##### 단지코드를 C2085, C1397 => N2085로 변경

In [42]:
train.loc[ train['단지코드']=='C2085',  "단지코드" ] = 'N2085'
train.loc[ train['단지코드']=='C1397',  "단지코드" ] = 'N2085'

In [43]:
train.loc[ train['단지코드']=='N2085', : ].shape

(14, 15)

##### 오류 코드 변경

##### C2431, C1649의 총세대수를 1047로 변경
##### C2431, C1649의 등록차량대수를 1214로 변경
##### C2431, C1649의 단지코드를 N2431로 변경

In [50]:
a = train.loc[ train['단지코드']=='C2431', : ]
b = train.loc[ train['단지코드']=='C1649', : ]

print(  a.shape, b.shape )
print( a['총세대수'], b['총세대수'])
print( a['등록차량수'], b['등록차량수'])

(2, 15) (4, 15)
2372    472
2373    472
Name: 총세대수, dtype: int64 2315    575
2316    575
2317    575
2318    575
Name: 총세대수, dtype: int64
2372    359.0
2373    359.0
Name: 등록차량수, dtype: float64 2315    855.0
2316    855.0
2317    855.0
2318    855.0
Name: 등록차량수, dtype: float64


In [51]:
train.loc[ train['단지코드']=='C2431',  "총세대수" ] = 1047
train.loc[ train['단지코드']=='C1649',  "총세대수" ] = 1047

train.loc[ train['단지코드']=='C2431',  "등록차량수" ] = 1214
train.loc[ train['단지코드']=='C1649',  "등록차량수" ] = 1214

train.loc[ train['단지코드']=='C2431',  "단지코드" ] = 'N2431'
train.loc[ train['단지코드']=='C1649',  "단지코드" ] = 'N2431'

In [52]:
train.loc[ train['단지코드']=='N2431', : ].shape

(6, 15)

##### C1036의 총세대수를 1243로 변경

##### C1036의 단지코드를 N1036로 변경

In [53]:
a = train.loc[ train['단지코드']=='C2675', : ]
b = train.loc[ train['단지코드']=='C1036', : ]
a.shape, b.shape

((0, 15), (7, 15))

In [54]:
train.loc[ train['단지코드']=='C1036',  "총세대수" ] = 1243
train.loc[ train['단지코드']=='C1036',  "단지코드" ] = 'N1036'

In [55]:
train.loc[ train['단지코드']=='N1036', : ].shape

(7, 15)